In [1]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import praw

import pickle

from markdown import markdown


from bs4 import BeautifulSoup as bs

In [2]:
reddit = praw.Reddit(client_id='vKDJ34lnNRAt5Q',
                     client_secret='ap0vMAKpJ7ukjRAEQLCGltyA-so',
                     password='thisisapassword',
                     user_agent='testscript by /u/throwawaytsds',
                     username='throwawaytsds')

In [3]:
print(reddit.user.me())


throwawaytsds


# Collect $n$ submissions in political parties subreddits ('controversial')

In [ ]:
n = 10000


scraped = {}
for subreddit in ['democrats', 'republican']:
    submissions = reddit.subreddit(subreddit).controversial('all',limit=n)
    if subreddit not in scraped.keys():
        scraped[subreddit] = []
    for _ in tqdm(list(range(n))):
        try:
            submission = submissions.next()
            scraped[subreddit].append(submission)
        except StopIteration as e:
            print('end of iteration')
            break
        except Exception as e:
            print('something else happened')
            print(e)

In [88]:
for k in list(scraped.keys()):
    print(len(scraped[k]))

3806
2529


In [68]:
scraped['democrat'] = scraped['democrats']
del scraped['democrats']

In [69]:
np.save('scraped.npy', scraped)

In [5]:
scraped = np.load('scraped.npy')
scraped = scraped.item()
scraped.keys()

dict_keys(['republican', 'democrat'])

In [6]:
scraped['republican'][0]

Submission(id='zxcr3')

In [7]:
data = {
    'ups': [],
    'ratio': [],
    'title': [],
    'text': [],
    'nr_comments': [],
    'label': []
}

In [ ]:
i = 0
for label in scraped.keys():
    for sub in tqdm(scraped[label]):
        data['ups'].append(sub.ups)
        data['ratio'].append(sub.upvote_ratio)
        data['title'].append(sub.title)
        data['text'].append(sub.selftext)
        data['nr_comments'].append(sub.num_comments)
        data['label'].append(label)

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df.to_csv('scraped.pd')

In [9]:
df = pd.read_csv('scraped.pd',index_col=0)

In [26]:
df.sample(10)

,ups,ratio,title,text,nr_comments,label
194,12,0.55,Obama runs from responsibility yet again.,NaN,15,republican
1270,0,0.47,Why Precisely Is Bernie Sanders Against Repara...,NaN,19,democrat
254,0,0.49,Whoops: PolitiFact's Romney 'Lie of the Year' ...,NaN,22,republican
116,0,0.49,North Korea says it will launch long-range roc...,NaN,25,republican
1294,0,0.50,The Bernie Bros are out in full force harassin...,NaN,1,democrat
1302,0,0.46,Rant,"This will probably be down voted or removed, b...",19,democrat
1965,2,0.50,Financial titans behind slam on Clinton: Hedge...,NaN,1,democrat
432,0,0.48,This is Why White Progressives Are Racist,NaN,11,republican
1069,16,0.58,PolitiFact: Pro-life Scott Walker accused of a...,NaN,5,democrat
1356,1,0.52,Watch Hillary comfort a little girl who fears ...,NaN,10,democrat


## Cleaning Markdown to text

In [27]:
for i, row in df.iterrows():
    if not pd.isna(row['text']) and not pd.isnull(row['text']):
        html = markdown(row['text'])
        text = ''.join(bs(html).findAll(text=True))
        text = text.replace('\n',' ')
        df.loc[i, 'text'] = text
    

In [28]:
df.head(10)

,ups,ratio,title,text,nr_comments,label
0,0,0.48,Scumbag Obama,NaN,48,republican
1,23,0.53,K-pop star PSY once advocated ‘slowly and pain...,NaN,239,republican
2,0,0.50,"For the Sake of the Republican Party, Don't Vo...",NaN,14,republican
3,3,0.51,"The Obama Disaster: Part II or ""America is sti...",NaN,18,republican
4,4,0.51,"LOL - response to Obama ""Lose Your Virginity f...",NaN,34,republican
5,0,0.50,"""The Science is Settled""",NaN,47,republican
6,0,0.48,What has caused more damage?,NaN,36,republican
7,0,0.50,What do you have to lose?! They’ve taken EVERY...,NaN,9,republican
8,2,0.51,The Heart & Character: How Mitt Romney shut do...,NaN,17,republican
9,0,0.50,Why I vote Democrat....,I voted Democrat because I believe oil compa...,37,republican


In [32]:
[('\n' in i[:-1], i) for i in df.text.sample(10).values if not pd.isna(i)]

[(False,
  "So this Covington situation has clearly shown the division in this country. I propose that the dems co-opt the MAGA with a blue hat. You want teams then make your own. See which hat makes the country better. Here is the catch though. Actually go out into your community and help. Don't donate money. Donate something much more valuable; your time.  Dont post about it on social media. Go get your hands dirty with your neighbors, but let's see which hat can do a better job. Red vs Blue put on a purple hat whatever. Once you put it on you are in service to your community mode. Fight over who can build a playground better. Who can deliver more food to the elderly. Firing off angry tweets about what a hat means to you doesn't do much. Take whatever issue you care about, and engage your energy in work. This is the country I would like to see.")]

In [33]:
text_dump = {
    
}

for i, row in df.iterrows():
    if row['label'] not in text_dump.keys():
        text_dump[row['label']] = ''

    text_dump[row['label']] += ' ' + row['title']
    
    if not pd.isna(row['text']) and not pd.isnull(row['text']):
        text_dump[row['label']] += ' ' + row['text']
        
    text_dump[row['label']] += ' \n '

In [34]:
len(text_dump['republican']), len(text_dump['democrat'])

(148364, 137692)

In [38]:
for label in text_dump.keys():
    with open('dump_%s.txt' %label, 'w', encoding='utf8') as f:
        f.write(text_dump[label])

In [37]:
pickle.dump(text_dump, open('text_dumps.pkl','wb'))